<a href="https://colab.research.google.com/github/wjh818/Data_hwj/blob/main/Deep_Learning/6_%EC%86%90%EC%8B%A4_%ED%95%A8%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 평균 제곱 오차 ( MSE - Mean Squared Error )
$$
E = \frac{1}{2}\sum_{k}(y_k-t_k)^2
$$

앞쪽이 $\frac{1}{2}$인 이유는 미분 했을 때 남는 식이 $(y_k-t_k)$ 이기 때문.
* $y_k$는 예측값을 의미한다. ($\hat y_k$)
* $t_k$는 정답(타깃값)을 의미한다.

In [1]:
import numpy as np
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

* `y`는 $softmax$의 결과물로써, 정답이 2일 확률을 60%로 추정하고 있다.
* `t`는 One-Hot Encoding 된 정답 레이블로써, 원래 정답이 2라고 정해져 있다.

In [2]:
def mean_squared_error(y, t):
  return 0.5 * np.sum((y-t) ** 2)

In [3]:
print("정답을 2로 추정했을 때의 MSE 값 : {:.3f}".format(mean_squared_error(np.array(y), np.array(t))))

정답을 2로 추정했을 때의 MSE 값 : 0.098


In [4]:
# 일부로 오답 예측을 만들어서 MSE 값을 측정하기. 예측값을 7이라고 가정한 경우로 테스트
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print("정답을 7로 추정했을 때의 MSE 값 : {:.3f}".format(mean_squared_error(np.array(y), np.array(t))))

정답을 7로 추정했을 때의 MSE 값 : 0.598


# 교차 엔트로피 오차 ( CEE - Cross Entropy Error )

$$
E = -\sum_{k}t_klog\;y_k
$$

One Hot Encoding된 $t_k$가 곱해지기 때문에, 정답이 아닌 타깃은 신경을 전혀 쓰지 않음. (곱하기 0)

즉, 예측값 자체가 오차 출력값이 된다.

In [5]:
def cross_entropy_error(y, t):
  delta = 1e-7
  return -np.sum(t * np.log(y + delta))

아주 작은 값인 $delta$를 더하는 이유는 `np.log` 함수에 `0`이 대입되면 마이너스 무한대를 나타내게 되기 때문에 `y`에 아주 작은 값인 delta를 더해서  `np.log` 함수의 결과물이 마이너스 무한대가 되는 것을 방지한다.

In [6]:
# 정답은 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 정답이 2일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print("정답을 2로 추정했을 때의 CEE값 : {:.3f}".format(cross_entropy_error(np.array(y), np.array(t))))

# 정답이 7일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print("정답을 7로 추정했을 때의 CEE값 : {:.3f}".format(cross_entropy_error(np.array(y), np.array(t))))

정답을 2로 추정했을 때의 CEE값 : 0.511
정답을 7로 추정했을 때의 CEE값 : 2.303


# 미니배치 학습
$$
E = -\frac{1}{N}\sum_n\;\sum_{k}t_{nk}log\;y_{nk}
$$

$N$개의 배치 데이터를 활용했을 때 바뀐 CEE 공식

In [7]:
# mnist 데이터셋 로딩
from tensorflow.keras import datasets
mnist = datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [8]:
X_train.shape

(60000, 28, 28)

In [9]:
y_train.shape

(60000,)

Cross Entropy는 One Hot Encoding 이 되어 있어야 함

* `X_train.shape -> (60000, 784)`
* `y_train.shape -> (60000, 10)`

In [10]:
# step 1 : reshape 부터 진행 -> (60000, 1)
y_train_step_1 = y_train.reshape(-1, 1)
y_train_step_1.shape

(60000, 1)

In [11]:
y_train_step_1[:3]

array([[5],
       [0],
       [4]], dtype=uint8)

In [12]:
# OneHotEncoder를 이용해서 원핫 인코딩 수행
from sklearn.preprocessing import OneHotEncoder

y_train_one_hot = OneHotEncoder().fit_transform(y_train_step_1)
y_train_one_hot = y_train_one_hot.toarray()

y_train_one_hot.shape

(60000, 10)

In [13]:
y_train_one_hot[:3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [14]:
X_train = X_train.reshape(60000, -1)
X_train.shape

(60000, 784)

## 미니배치 구현하기

In [15]:
# 훈련 데이터에서 무작위로 10장만 빼내오기
train_size = X_train.shape[0] # 전체 훈련 데이터셋 크기
batch_size = 10 # 미니 배치의 사이즈
batch_mask = np.random.choice(train_size, batch_size) # train_size에서 무작위로 batch_size만큼의 정수를 선택

X_batch = X_train[batch_mask] # 랜덤으로 선택된 인덱스에 있는 데이터만 추려내기
y_batch = y_train_one_hot[batch_mask] # 원핫 인코딩된 y_train에서 선택된 인덱스에 있는 데이터만 추려내기

In [16]:
print("무작위로 선택된 인덱스 : {}".format(batch_mask))

무작위로 선택된 인덱스 : [ 1764 23474 57514 51686 13671 58648  1254 13395 47693 37857]


배치용 교차 엔트로피
$$
E = -\frac{1}{N}\sum_n\;\sum_{k}t_{nk}log\;y_{nk}
$$

In [17]:
# 배치 및 배치가 아닐 때 까지 고려
def cross_entropy_error(y, t):

  # 1차 원일 때에 대한 처리( 배치가 아닐 때의 처리 )
  if y.ndim == 1:
    # 강제로 2차원 배열화 시키는 것
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  
  batch_size = y.shape[0]
  return -np.sum(t * np.log(y)) / batch_size

In [18]:
# 원-핫 인코딩이 되어있지 않은 경우 대응
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [19]:
arr = np.arange(10, 100)
arr

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94,
       95, 96, 97, 98, 99])

In [20]:
arr[[1, 3, 6, 0, 5]]

array([11, 13, 16, 10, 15])